In [1]:
import pandas as pd
import numpy as np
from matplotlib import pyplot as plt
import os
from datetime import datetime, timedelta

# Variables

In [2]:
# BUS Network
bus_network_list = ['basic', 'diagonal', 'ring', 'mature']
# Train Headway (min)
train_headway_list = [i for i in range(10, 31, 5)]

# Network

In [3]:
# Load Network Files
network_name = "11-500"
network_path = f"data/network/{network_name}/"
node_df = pd.read_csv(network_path + "nodes.csv")
link_df = pd.read_csv(network_path + "edges.csv")

node_id_list = node_df['node_index'].tolist()
print(f"Number of nodes: {len(node_id_list)}")

left_node_id_list = [i for i in range(0, len(node_id_list)//2)]
right_node_id_list = [i for i in range(len(node_id_list)//2, len(node_id_list))]
print(f"Number of left nodes: {len(left_node_id_list)}")
print(f"Number of right nodes: {len(right_node_id_list)}")

left_mobility_node_id_list = [116, 117, 118, 119, 120]
right_mobility_node_id_list = [237, 238, 239, 240, 241]

Number of nodes: 242
Number of left nodes: 121
Number of right nodes: 121


In [4]:
# Load Distance Matrix
distance_matrix = np.load(network_path + "dist_matrix.npy")
print(f"Distance matrix shape: {distance_matrix.shape}")

Distance matrix shape: (242, 242)


In [5]:
# Load Travel Time Matrix
tt_matrix = np.load(network_path + "tt_matrix.npy")
print(f"Travel time matrix shape: {tt_matrix.shape}")

Travel time matrix shape: (242, 242)


# Create General Data

In [6]:
general_output_path = "data/gtfs/general/"
if not os.path.exists(general_output_path):
    os.makedirs(general_output_path)

# Create stations_fp.csv
stations_fp_df = pd.DataFrame(columns=['station_id','station_name','station_lat','station_lon','stops_included','station_stop_transfer_times','num_stops_included'])
for index, node in node_df.iterrows():
    station_id = node['node_index']
    station_name = station_id
    station_lat = node['pos_y']
    station_lon = node['pos_x']

    stops_included = "['{}-0';'{}-1';'{}-2';'{}-3';'{}-4';'{}-5']".format(station_id, station_id, station_id, station_id, station_id, station_id)
    station_stop_transfer_times = '[0;0;0;0;0;0]'
    num_stops_included = 6

    new_row = pd.DataFrame({
        'station_id': [station_id],
        'station_name': [station_name],
        'station_lat': [station_lat],
        'station_lon': [station_lon],
        'stops_included': [stops_included],
        'station_stop_transfer_times': [station_stop_transfer_times],
        'num_stops_included': [num_stops_included]
    })

    stations_fp_df = pd.concat([stations_fp_df, new_row], ignore_index=True)

# Remove non-stop nodes
left_non_stops_node_id_list = [1,2,3,4,6,7,8,13,14,15,17,18,21,22,31,32,33,34,37,42,43,44,45,50,52,53,62,63,65,70,71,72,73,78,81,82,83,84,93,94,97,98,100,101,102,107,108,109,111,112,113,114]
right_non_stops_node_id_list = [i + 121 for i in left_non_stops_node_id_list]
non_stop_node_id_list = left_non_stops_node_id_list + right_non_stops_node_id_list

stations_fp_df = stations_fp_df[~stations_fp_df['station_id'].isin(non_stop_node_id_list)].reset_index(drop=True)

# Change 120 to A, 241 to B
stations_fp_df.loc[stations_fp_df['station_id'] == 120, 'station_id'] = 'A'
stations_fp_df.loc[stations_fp_df['station_id'] == 'A', 'station_name'] = 'A Hub'
stations_fp_df.loc[stations_fp_df['station_id'] == 'A', 'stops_included'] = "['A-0';'A-1';'A-2';'A-3';'A-4';'A-5']"

stations_fp_df.loc[stations_fp_df['station_id'] == 241, 'station_id'] = 'B'
stations_fp_df.loc[stations_fp_df['station_id'] == 'B', 'station_name'] = 'B Hub'
stations_fp_df.loc[stations_fp_df['station_id'] == 'B', 'stops_included'] = "['B-0';'B-1';'B-2';'B-3';'B-4';'B-5']"

# Save stations_fp.txt
stations_fp_df.to_csv(general_output_path + "stations_fp.txt", index=False)

In [7]:
# Create stops_fp.txt
stops_fp_df = pd.DataFrame(columns=['stop_id'])
for index, row in stations_fp_df.iterrows():
    # Convert stops_included string to list
    s = row['stops_included']
    if isinstance(s, str):
        s_clean = s.strip()
        if s_clean.startswith('[') and s_clean.endswith(']'):
            s_clean = s_clean[1:-1]
        s_clean = s_clean.replace("'", "").replace('"', '').strip()
        if ';' in s_clean:
            parts = [p.strip() for p in s_clean.split(';') if p.strip()]
        elif ',' in s_clean:
            parts = [p.strip() for p in s_clean.split(',') if p.strip()]
        elif s_clean == '':
            parts = []
        else:
            parts = [s_clean]
    else:
        parts = list(s) if hasattr(s, '__iter__') and not isinstance(s, str) else [s]
    row['stops_included'] = ';'.join(parts)
    stop_ids = row['stops_included']
    for stop_id in stop_ids.split(';'):
        stops_fp_df = pd.concat([stops_fp_df, pd.DataFrame({'stop_id': [stop_id]})], ignore_index=True)
stops_fp_df.to_csv(os.path.join(general_output_path, "stops_fp.txt"), index=False)

In [8]:
# Prepare agency_fp.txt
agency_df = pd.DataFrame({
    'agency_id': [0, 1, 2, 3],
    'agency_name': ['intercity-express', 'bus-basic', 'bus-ring', 'bus-diagonal'],
})
agency_df.to_csv(os.path.join(general_output_path, "agency_fp.txt"), index=False)

In [9]:
# Create calendar_fp.txt
calendar_df = pd.DataFrame({
    'service_id': [0],
    'start_date': [20000101],
    'end_date': [20991231],
    'monday': [1],
    'tuesday': [1],
    'wednesday': [1],
    'thursday': [1],
    'friday': [1],
    'saturday': [1],
    'sunday': [1],
})
calendar_df.to_csv(os.path.join(general_output_path, "calendar_fp.txt"), index=False)

In [10]:
# Create street_station_transfers_fp.txt
all_station_ids = stations_fp_df['station_id'].tolist()

street_station_transfers_fp_df = pd.DataFrame({
    'node_id': all_station_ids,
    'closest_station_id': all_station_ids,
    'street_station_transfer_time': 60
})

# Change node id from A and B back to 120 and 241
street_station_transfers_fp_df.loc[street_station_transfers_fp_df['node_id'] == 'A', 'node_id'] = 120
street_station_transfers_fp_df.loc[street_station_transfers_fp_df['node_id'] == 'B', 'node_id'] = 241

street_station_transfers_fp_df.to_csv(os.path.join(general_output_path, "street_station_transfers_fp.txt"), index=False)

In [11]:
# Create transfers_fp.txt
# For each station, the transfer time between its stops is 30 seconds
transfers_fp_df = pd.DataFrame(columns=['from_stop_id', 'to_stop_id', 'min_transfer_time'])
for index, row in stations_fp_df.iterrows():
    stops_included = row['stops_included'].split(';')
    for i in range(len(stops_included)):
        for j in range(len(stops_included)):
            if i != j:
                new_row = pd.DataFrame({
                    'from_stop_id': [stops_included[i]],
                    'to_stop_id': [stops_included[j]],
                    'min_transfer_time': [30]
                })
                transfers_fp_df = pd.concat([transfers_fp_df, new_row], ignore_index=True)
transfers_fp_df.to_csv(os.path.join(general_output_path, "transfers_fp.txt"), index=False)

In [12]:
def expand_timetable(base_trip_data, study_start_str, study_end_str, headway_minutes):
    """
    扩展单个基准行程的时刻表。

    参数:
    - base_trip_data (dict): 包含基准行程信息的字典。
    - study_start_str (str): 研究范围的开始时间 (HH:MM:SS)。
    - study_end_str (str): 研究范围的结束时间 (HH:MM:SS)。
    - headway_minutes (int): 发车间隔（分钟）。

    返回:
    - pd.DataFrame: 包含所有扩展行程的 DataFrame。
    """
    # 将时间字符串转换为 datetime 对象以便计算
    study_start_time = datetime.strptime(study_start_str, '%H:%M:%S')
    study_end_time = datetime.strptime(study_end_str, '%H:%M:%S')
    headway = timedelta(minutes=headway_minutes)

    # 将基准行程字典转换为 DataFrame
    base_df = pd.DataFrame(base_trip_data)
    
    # 将 DataFrame 中的时间字符串转换为 datetime 对象
    base_df['arrival_time'] = pd.to_datetime(base_df['arrival_time'], format='%H:%M:%S')
    base_df['departure_time'] = pd.to_datetime(base_df['departure_time'], format='%H:%M:%S')

    all_trips = [base_df]
    base_trip_id = base_df['trip_id'].iloc[0]

    # --- 向前扩展行程 ---
    current_trip_df = base_df.copy()
    counter = 1
    while True:
        next_trip_df = current_trip_df.copy()
        # 增加 headway 时间
        next_trip_df['arrival_time'] += headway
        next_trip_df['departure_time'] += headway
        
        # 检查新行程的开始时间是否在研究范围内
        if next_trip_df['departure_time'].iloc[0] >= study_end_time:
            break
            
        # 更新 trip_id
        next_trip_df['trip_id'] = f"{base_trip_id}-{counter}"
        all_trips.append(next_trip_df)
        
        current_trip_df = next_trip_df
        counter += 1

    # --- 向后扩展行程 ---
    current_trip_df = base_df.copy()
    while True:
        prev_trip_df = current_trip_df.copy()
        # 减去 headway 时间
        prev_trip_df['arrival_time'] -= headway
        prev_trip_df['departure_time'] -= headway

        # 检查新行程的开始时间是否在研究范围内
        if prev_trip_df['departure_time'].iloc[0] < study_start_time:
            break

        # 更新 trip_id
        prev_trip_df['trip_id'] = f"{base_trip_id}-{counter}"
        # 将向后生成的行程插入到列表的开头，以保持时间顺序
        all_trips.insert(0, prev_trip_df)
        
        current_trip_df = prev_trip_df
        counter += 1
        
    # 合并这个基准线路产生的所有行程
    expanded_df = pd.concat(all_trips, ignore_index=True)

    # Add 0 to trip_id for base trip
    expanded_df.loc[expanded_df['trip_id'] == base_trip_id, 'trip_id'] = f"{base_trip_id}-0"
    return expanded_df

# Create GTFS for Basic Bus Network

In [13]:
BUS_HEADWAY = 10
STUDY_START = "00:00:00"
STUDY_END = "08:00:00"

# Prepare output path
basic_output_path = f"data/gtfs/bus/basic/bus_headway_{BUS_HEADWAY}/"
if not os.path.exists(basic_output_path):
    os.makedirs(basic_output_path)

# Create routes_fp.txt
routes_df = pd.DataFrame({
    'route_id': [1, 2, 3, 4],
    'route_short_name': ['bus-basic-we-left', 'bus-basic-ns-left', 'bus-basic-we-right', 'bus-basic-ns-right'],
    'route_desc': ['bus', 'bus', 'bus', 'bus'],
})
routes_df.to_csv(os.path.join(basic_output_path, "routes_fp.txt"), index=False)

In [14]:
# Create stop_times_fp.txt
# Use 10 minute headway
stop_times_fp_df = pd.DataFrame(columns=['trip_id', 'arrival_time', 'departure_time', 'stop_id', 'stop_sequence'])

# For w-e direction in the left side
we_left_base_trip_times_0 = {
    'trip_id': '1-0', # route_id-direction
    'stop_id': ['54-1', '55-1', '56-1', '57-1', '117-1', 'A-1', '118-1', '58-1', '59-1', '60-1', '61-1'],
    'arrival_time': ['00:54:30', '00:56:00', '00:57:30', '00:59:00', '01:00:30', '01:02:00', '01:03:30', '01:05:00', '01:06:30', '01:08:00', '01:09:30'],
    'departure_time': ['00:54:30', '00:56:00', '00:57:30', '00:59:00', '01:00:30', '01:02:00', '01:03:30', '01:05:00', '01:06:30', '01:08:00', '01:09:30'],
    'stop_sequence': [1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11]
}

we_left_base_trip_times_1 = {
    'trip_id': '1-1', # route_id-direction
    'stop_id': ['61-1', '60-1', '59-1', '58-1', '118-1', 'A-1', '117-1', '57-1', '56-1', '55-1', '54-1'],
    'arrival_time': ['00:54:30', '00:56:00', '00:57:30', '00:59:00', '01:00:30', '01:02:00', '01:03:30', '01:05:00', '01:06:30', '01:08:00', '01:09:30'],
    'departure_time': ['00:54:30', '00:56:00', '00:57:30', '00:59:00', '01:00:30', '01:02:00', '01:03:30', '01:05:00', '01:06:30', '01:08:00', '01:09:30'],
    'stop_sequence': [1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11]
}

ns_left_base_trip_times_0 = {
    'trip_id': '2-0', # route_id-direction
    'stop_id': ['110-2', '99-2', '88-2', '77-2', '119-2', 'A-2', '116-2', '38-2', '27-2', '16-2', '5-2'],
    'arrival_time': ['00:59:30', '01:01:00', '01:02:30', '01:04:00', '01:05:30', '01:07:00', '01:08:30', '01:10:00', '01:11:30', '01:13:00', '01:14:30'],
    'departure_time': ['00:59:30', '01:01:00', '01:02:30', '01:04:00', '01:05:30', '01:07:00', '01:08:30', '01:10:00', '01:11:30', '01:13:00', '01:14:30'],
    'stop_sequence': [1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11]
}

ns_left_base_trip_times_1 = {
    'trip_id': '2-1', # route_id-direction
    'stop_id': ['5-2', '16-2', '27-2', '38-2', '116-2', 'A-2', '119-2', '77-2', '88-2', '99-2', '110-2'],
    'arrival_time': ['00:59:30', '01:01:00', '01:02:30', '01:04:00', '01:05:30', '01:07:00', '01:08:30', '01:10:00', '01:11:30', '01:13:00', '01:14:30'],
    'departure_time': ['00:59:30', '01:01:00', '01:02:30', '01:04:00', '01:05:30', '01:07:00', '01:08:30', '01:10:00', '01:11:30', '01:13:00', '01:14:30'],
    'stop_sequence': [1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11]
}

we_right_base_trip_times_0 = {
    'trip_id': '3-0', # route_id-direction
    'stop_id': ['175-1', '176-1', '177-1', '178-1', '238-1', 'B-1', '239-1', '179-1', '180-1', '181-1', '182-1'],
    'arrival_time': ['00:54:30', '00:56:00', '00:57:30', '00:59:00', '01:00:30', '01:02:00', '01:03:30', '01:05:00', '01:06:30', '01:08:00', '01:09:30'],
    'departure_time': ['00:54:30', '00:56:00', '00:57:30', '00:59:00', '01:00:30', '01:02:00', '01:03:30', '01:05:00', '01:06:30', '01:08:00', '01:09:30'],
    'stop_sequence': [1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11]
}

we_right_base_trip_times_1 = {
    'trip_id': '3-1', # route_id-direction
    'stop_id': ['182-1', '181-1', '180-1', '179-1', '239-1', 'B-1', '238-1', '178-1', '177-1', '176-1', '175-1'],
    'arrival_time': ['00:54:30', '00:56:00', '00:57:30', '00:59:00', '01:00:30', '01:02:00', '01:03:30', '01:05:00', '01:06:30', '01:08:00', '01:09:30'],
    'departure_time': ['00:54:30', '00:56:00', '00:57:30', '00:59:00', '01:00:30', '01:02:00', '01:03:30', '01:05:00', '01:06:30', '01:08:00', '01:09:30'],
    'stop_sequence': [1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11]
}

ns_right_base_trip_times_0 = {
    'trip_id': '4-0', # route_id-direction
    'stop_id': ['231-2', '220-2', '209-2', '198-2', '240-2', 'B-2', '237-2', '159-2', '148-2', '137-2', '126-2'],
    'arrival_time': ['00:59:30', '01:01:00', '01:02:30', '01:04:00', '01:05:30', '01:07:00', '01:08:30', '01:10:00', '01:11:30', '01:13:00', '01:14:30'],
    'departure_time': ['00:59:30', '01:01:00', '01:02:30', '01:04:00', '01:05:30', '01:07:00', '01:08:30', '01:10:00', '01:11:30', '01:13:00', '01:14:30'],
    'stop_sequence': [1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11]
}

ns_right_base_trip_times_1 = {
    'trip_id': '4-1', # route_id-direction
    'stop_id': ['126-2', '137-2', '148-2', '159-2', '237-2', 'B-2', '240-2', '198-2', '209-2', '220-2', '231-2'],
    'arrival_time': ['00:59:30', '01:01:00', '01:02:30', '01:04:00', '01:05:30', '01:07:00', '01:08:30', '01:10:00', '01:11:30', '01:13:00', '01:14:30'],
    'departure_time': ['00:59:30', '01:01:00', '01:02:30', '01:04:00', '01:05:30', '01:07:00', '01:08:30', '01:10:00', '01:11:30', '01:13:00', '01:14:30'],
    'stop_sequence': [1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11]
}

# Use headway to create multiple trips for the study period: 00:00:00 to 08:00:00
base_schedules = [we_left_base_trip_times_0,
                    we_left_base_trip_times_1,
                    ns_left_base_trip_times_0,
                    ns_left_base_trip_times_1,
                    we_right_base_trip_times_0,
                    we_right_base_trip_times_1,
                    ns_right_base_trip_times_0,
                    ns_right_base_trip_times_1]

all_schedules_list = []
for schedule_data in base_schedules:
    expanded_schedule = expand_timetable(schedule_data, STUDY_START, STUDY_END, BUS_HEADWAY)
    all_schedules_list.append(expanded_schedule)

final_timetable = pd.concat(all_schedules_list, ignore_index=True)

final_timetable['arrival_time'] = final_timetable['arrival_time'].dt.strftime('%H:%M:%S')
final_timetable['departure_time'] = final_timetable['departure_time'].dt.strftime('%H:%M:%S')

stop_times_fp_df = final_timetable
stop_times_fp_df.to_csv(os.path.join(basic_output_path, "stop_times_fp.txt"), index=False)

In [15]:
# Create trips_fp.txt
trips_fp_df = pd.DataFrame(columns=['trip_id','route_id','service_id','direction_id'])

all_trip_ids = final_timetable['trip_id'].unique().tolist()
for trip_id in all_trip_ids:
    route_id = int(trip_id.split('-')[0])
    direction_id = int(trip_id.split('-')[1])
    new_row = pd.DataFrame({
        'trip_id': [trip_id],
        'route_id': [route_id],
        'service_id': [0],
        'direction_id': [direction_id]
    })
    trips_fp_df = pd.concat([trips_fp_df, new_row], ignore_index=True)

trips_fp_df.to_csv(os.path.join(basic_output_path, "trips_fp.txt"), index=False)

# Create GTFS for Ring Bus Network

In [25]:
BUS_HEADWAY = 10
STUDY_START = "00:00:00"
STUDY_END = "08:00:00"

# Prepare output path
basic_output_path = f"data/gtfs/bus/ring/bus_headway_{BUS_HEADWAY}/"
if not os.path.exists(basic_output_path):
    os.makedirs(basic_output_path)

# Create routes_fp.txt
routes_df = pd.DataFrame({
    'route_id': [5, 6],
    'route_short_name': ['bus-ring-left', 'bus-ring-right'],
    'route_desc': ['bus', 'bus'],
})
routes_df.to_csv(os.path.join(basic_output_path, "routes_fp.txt"), index=False)

In [26]:
# Create stop_times_fp.txt
# Use 10 minute headway
stop_times_fp_df = pd.DataFrame(columns=['trip_id', 'arrival_time', 'departure_time', 'stop_id', 'stop_sequence'])

left_ring_trip_times_0 = {
    'trip_id': '5-0', # route_id-direction
    'stop_id': ['56-3', '46-3', '35-3', '24-3', '25-3', '26-3', '27-3', '28-3', '29-3', '30-3', '41-3', '51-3', '59-3', '69-3', '80-3', '91-3', '90-3', '89-3', '88-3', '87-3', '86-3', '85-3', '74-3', '64-3', '56-3'],
    'arrival_time': ['01:09:20', '01:11:00', '01:12:40', '01:14:20', '01:16:00', '01:17:40', '01:19:20', '01:21:00', '01:22:40', '01:24:20', '01:26:00', '01:27:40', '01:29:20', '01:31:00', '01:32:40', '01:34:20', '01:36:00', '01:37:40', '01:39:20', '01:41:00', '01:42:40', '01:44:20', '01:46:00', '01:47:40', '01:49:20'],
    'departure_time': ['01:09:30', '01:11:10', '01:12:50', '01:14:30', '01:16:10', '01:17:50', '01:19:30', '01:21:10', '01:22:50', '01:24:30', '01:26:10', '01:27:50', '01:29:30', '01:31:10', '01:32:50', '01:34:30', '01:36:10', '01:37:50', '01:39:30', '01:41:10', '01:42:50', '01:44:30', '01:46:10', '01:47:50', '01:49:20'],
    'stop_sequence': [1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25]
}

left_ring_trip_times_1 = {
    'trip_id': '5-1', # route_id-direction
    'stop_id': ['56-3', '64-3', '74-3', '85-3', '86-3', '87-3', '88-3', '89-3', '90-3', '91-3', '80-3', '69-3', '59-3', '51-3', '41-3', '30-3', '29-3', '28-3', '27-3', '26-3', '25-3', '24-3', '35-3', '46-3', '56-3'],
    'arrival_time': ['01:09:20', '01:11:00', '01:12:40', '01:14:20', '01:16:00', '01:17:40', '01:19:20', '01:21:00', '01:22:40', '01:24:20', '01:26:00', '01:27:40', '01:29:20', '01:31:00', '01:32:40', '01:34:20', '01:36:00', '01:37:40', '01:39:20', '01:41:00', '01:42:40', '01:44:20', '01:46:00', '01:47:40', '01:49:20'],
    'departure_time': ['01:09:30', '01:11:10', '01:12:50', '01:14:30', '01:16:10', '01:17:50', '01:19:30', '01:21:10', '01:22:50', '01:24:30', '01:26:10', '01:27:50', '01:29:30', '01:31:10', '01:32:50', '01:34:30', '01:36:10', '01:37:50', '01:39:30', '01:41:10', '01:42:50', '01:44:30', '01:46:10', '01:47:50', '01:49:20'],
    'stop_sequence': [1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25]
}

right_ring_trip_times_0 = {
    'trip_id': '6-0', # route_id-direction
    'stop_id': ['177-3', '167-3', '156-3', '145-3', '146-3', '147-3', '148-3', '149-3', '150-3', '151-3', '162-3', '172-3', '180-3', '190-3', '201-3', '212-3', '211-3', '210-3', '209-3', '208-3', '207-3', '206-3', '195-3', '185-3', '177-3'],
    'arrival_time': ['01:09:20', '01:11:00', '01:12:40', '01:14:20', '01:16:00', '01:17:40', '01:19:20', '01:21:00', '01:22:40', '01:24:20', '01:26:00', '01:27:40', '01:29:20', '01:31:00', '01:32:40', '01:34:20', '01:36:00', '01:37:40', '01:39:20', '01:41:00', '01:42:40', '01:44:20', '01:46:00', '01:47:40', '01:49:20'],
    'departure_time': ['01:09:30', '01:11:10', '01:12:50', '01:14:30', '01:16:10', '01:17:50', '01:19:30', '01:21:10', '01:22:50', '01:24:30', '01:26:10', '01:27:50', '01:29:30', '01:31:10', '01:32:50', '01:34:30', '01:36:10', '01:37:50', '01:39:30', '01:41:10', '01:42:50', '01:44:30', '01:46:10', '01:47:50', '01:49:20'],
    'stop_sequence': [1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25]
}

right_ring_trip_times_1 = {
    'trip_id': '6-1', # route_id-direction
    'stop_id': ['177-3','185-3', '195-3', '206-3', '207-3', '208-3', '209-3', '210-3', '211-3', '212-3', '201-3', '190-3', '180-3', '172-3', '162-3', '151-3', '150-3', '149-3', '148-3', '147-3', '146-3', '145-3', '156-3', '167-3', '177-3'],
    'arrival_time': ['01:09:20', '01:11:00', '01:12:40', '01:14:20', '01:16:00', '01:17:40', '01:19:20', '01:21:00', '01:22:40', '01:24:20', '01:26:00', '01:27:40', '01:29:20', '01:31:00', '01:32:40', '01:34:20', '01:36:00', '01:37:40', '01:39:20', '01:41:00', '01:42:40', '01:44:20', '01:46:00', '01:47:40', '01:49:20'],
    'departure_time': ['01:09:30', '01:11:10', '01:12:50', '01:14:30', '01:16:10', '01:17:50', '01:19:30', '01:21:10', '01:22:50', '01:24:30', '01:26:10', '01:27:50', '01:29:30', '01:31:10', '01:32:50', '01:34:30', '01:36:10', '01:37:50', '01:39:30', '01:41:10', '01:42:50', '01:44:30', '01:46:10', '01:47:50', '01:49:20'],
    'stop_sequence': [1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25]
}

# Use headway to create multiple trips for the study period: 00:00:00 to 08:00:00
base_schedules = [
    left_ring_trip_times_0,
    left_ring_trip_times_1,
    right_ring_trip_times_0,
    right_ring_trip_times_1
]

all_schedules_list = []
for schedule_data in base_schedules:
    expanded_schedule = expand_timetable(schedule_data, STUDY_START, STUDY_END, BUS_HEADWAY)
    all_schedules_list.append(expanded_schedule)
final_timetable = pd.concat(all_schedules_list, ignore_index=True)
final_timetable['arrival_time'] = final_timetable['arrival_time'].dt.strftime('%H:%M:%S')
final_timetable['departure_time'] = final_timetable['departure_time'].dt.strftime('%H:%M:%S')
stop_times_fp_df = final_timetable
stop_times_fp_df.to_csv(os.path.join(basic_output_path, "stop_times_fp.txt"), index=False)


In [27]:
# Create trips_fp.txt
trips_fp_df = pd.DataFrame(columns=['trip_id','route_id','service_id','direction_id'])

all_trip_ids = final_timetable['trip_id'].unique().tolist()
for trip_id in all_trip_ids:
    route_id = int(trip_id.split('-')[0])
    direction_id = int(trip_id.split('-')[1])
    new_row = pd.DataFrame({
        'trip_id': [trip_id],
        'route_id': [route_id],
        'service_id': [0],
        'direction_id': [direction_id]
    })
    trips_fp_df = pd.concat([trips_fp_df, new_row], ignore_index=True)

trips_fp_df.to_csv(os.path.join(basic_output_path, "trips_fp.txt"), index=False)

# Create GTFS for Diagonal Bus Network

In [19]:
BUS_HEADWAY = 10
STUDY_START = "00:00:00"
STUDY_END = "08:00:00"

# Prepare output path
basic_output_path = f"data/gtfs/bus/diagonal/bus_headway_{BUS_HEADWAY}/"
if not os.path.exists(basic_output_path):
    os.makedirs(basic_output_path)

# Create routes_fp.txt
routes_df = pd.DataFrame({
    'route_id': [7, 8, 9, 10],
    'route_short_name': ['bus-diagonal-sw-left', 'bus-diagonal-se-left', 'bus-diagonal-sw-right', 'bus-diagonal-se-right'],
    'route_desc': ['bus', 'bus', 'bus', 'bus'],
})
routes_df.to_csv(os.path.join(basic_output_path, "routes_fp.txt"), index=False)

In [20]:
# Create stop_times_fp.txt
# Use 10 minute headway
stop_times_fp_df = pd.DataFrame(columns=['trip_id', 'arrival_time', 'departure_time', 'stop_id', 'stop_sequence'])

sw_left_diagonal_trip_times_0 = {
    'trip_id': '7-0', # route_id-direction
    'stop_id': ['0-4', '11-4', '12-4', '23-4', '24-4', '35-4', '36-4', '47-4', '48-4', '117-4', 'A-4', '118-4', '67-4', '68-4', '79-4', '80-4', '91-4', '92-4', '103-4', '104-4', '115-4'],
    'arrival_time': ['00:48:00', '00:49:30', '00:51:00', '00:52:30', '00:54:00', '00:55:30', '00:57:00', '00:58:30', '01:00:00', '01:01:30', '01:03:00', '01:04:30', '01:06:00', '01:07:30', '01:09:00', '01:10:30', '01:12:00', '01:13:30', '01:15:00', '01:16:30', '01:18:00'],
    'departure_time': ['00:48:00', '00:49:30', '00:51:00', '00:52:30', '00:54:00', '00:55:30', '00:57:00', '00:58:30', '01:00:00', '01:01:30', '01:03:00', '01:04:30', '01:06:00', '01:07:30', '01:09:00', '01:10:30', '01:12:00', '01:13:30', '01:15:00', '01:16:30', '01:18:00'],
    'stop_sequence': [1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21]
}

sw_left_diagonal_trip_times_1 = {
    'trip_id': '7-1', # route_id-direction
    'stop_id': ['115-4', '104-4', '103-4', '92-4', '91-4', '80-4', '79-4', '68-4', '67-4', '118-4', 'A-4', '117-4', '48-4', '47-4', '36-4', '35-4', '24-4', '23-4', '12-4', '11-4', '0-4'],
    'arrival_time': ['00:48:00', '00:49:30', '00:51:00', '00:52:30', '00:54:00', '00:55:30', '00:57:00', '00:58:30', '01:00:00', '01:01:30', '01:03:00', '01:04:30', '01:06:00', '01:07:30', '01:09:00', '01:10:30', '01:12:00', '01:13:30', '01:15:00', '01:16:30', '01:18:00'],
    'departure_time': ['00:48:00', '00:49:30', '00:51:00', '00:52:30', '00:54:00', '00:55:30', '00:57:00', '00:58:30', '01:00:00', '01:01:30', '01:03:00', '01:04:30', '01:06:00', '01:07:30', '01:09:00', '01:10:30', '01:12:00', '01:13:30', '01:15:00', '01:16:30', '01:18:00'],
    'stop_sequence': [1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21]
}

se_left_diagonal_trip_times_0 = {
    'trip_id': '8-0', # route_id-direction
    'stop_id': ['10-5', '9-5', '20-5', '19-5', '30-5', '29-5', '40-5', '39-5', '49-5', '116-5', 'A-5', '119-5', '66-5', '76-5', '75-5', '86-5', '85-5', '96-5', '95-5', '106-5', '105-5'],
    'arrival_time': ['00:53:00', '00:54:30', '00:56:00', '00:57:30', '00:59:00', '01:00:30', '01:02:00', '01:03:30', '01:05:00', '01:06:30', '01:08:00', '01:09:30', '01:11:00', '01:12:30', '01:14:00', '01:15:30', '01:17:00', '01:18:30', '01:20:00', '01:21:30', '01:23:00'],
    'departure_time': ['00:53:00', '00:54:30', '00:56:00', '00:57:30', '00:59:00', '01:00:30', '01:02:00', '01:03:30', '01:05:00', '01:06:30', '01:08:00', '01:09:30', '01:11:00', '01:12:30', '01:14:00', '01:15:30', '01:17:00', '01:18:30', '01:20:00', '01:21:30', '01:23:00'],
    'stop_sequence': [1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21]
}

se_left_diagonal_trip_times_1 = {
    'trip_id': '8-1', # route_id-direction
    'stop_id': ['105-5', '106-5', '95-5', '96-5', '85-5', '86-5', '75-5', '76-5', '66-5', '119-5', 'A-5', '116-5', '49-5', '39-5', '40-5', '29-5', '30-5', '19-5', '20-5', '9-5', '10-5'],
    'arrival_time': ['00:53:00', '00:54:30', '00:56:00', '00:57:30', '00:59:00', '01:00:30', '01:02:00', '01:03:30', '01:05:00', '01:06:30', '01:08:00', '01:09:30', '01:11:00', '01:12:30', '01:14:00', '01:15:30', '01:17:00', '01:18:30', '01:20:00', '01:21:30', '01:23:00'],
    'departure_time': ['00:53:00', '00:54:30', '00:56:00', '00:57:30', '00:59:00', '01:00:30', '01:02:00', '01:03:30', '01:05:00', '01:06:30', '01:08:00', '01:09:30', '01:11:00', '01:12:30', '01:14:00', '01:15:30', '01:17:00', '01:18:30', '01:20:00', '01:21:30', '01:23:00'],
    'stop_sequence': [1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21]
}

sw_right_diagonal_trip_times_0 = {
    'trip_id': '9-0', # route_id-direction
    'stop_id': ['121-4', '132-4', '133-4', '144-4', '145-4', '156-4', '157-4', '168-4', '169-4', '238-4', 'B-4', '239-4', '188-4', '189-4', '200-4', '201-4', '212-4', '213-4', '224-4', '225-4', '236-4'],
    'arrival_time': ['00:48:00', '00:49:30', '00:51:00', '00:52:30', '00:54:00', '00:55:30', '00:57:00', '00:58:30', '01:00:00', '01:01:30', '01:03:00', '01:04:30', '01:06:00', '01:07:30', '01:09:00', '01:10:30', '01:12:00', '01:13:30', '01:15:00', '01:16:30', '01:18:00'],
    'departure_time': ['00:48:00', '00:49:30', '00:51:00', '00:52:30', '00:54:00', '00:55:30', '00:57:00', '00:58:30', '01:00:00', '01:01:30', '01:03:00', '01:04:30', '01:06:00', '01:07:30', '01:09:00', '01:10:30', '01:12:00', '01:13:30', '01:15:00', '01:16:30', '01:18:00'],
    'stop_sequence': [1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21]
}

sw_right_diagonal_trip_times_1 = {
    'trip_id': '9-1', # route_id-direction
    'stop_id': ['236-4', '225-4', '224-4', '213-4', '212-4', '201-4', '200-4', '189-4', '188-4', '239-4', 'B-4', '238-4', '169-4', '168-4', '157-4', '156-4', '145-4', '144-4', '133-4', '132-4', '121-4'],
    'arrival_time': ['00:48:00', '00:49:30', '00:51:00', '00:52:30', '00:54:00', '00:55:30', '00:57:00', '00:58:30', '01:00:00', '01:01:30', '01:03:00', '01:04:30', '01:06:00', '01:07:30', '01:09:00', '01:10:30', '01:12:00', '01:13:30', '01:15:00', '01:16:30', '01:18:00'],
    'departure_time': ['00:48:00', '00:49:30', '00:51:00', '00:52:30', '00:54:00', '00:55:30', '00:57:00', '00:58:30', '01:00:00', '01:01:30', '01:03:00', '01:04:30', '01:06:00', '01:07:30', '01:09:00', '01:10:30', '01:12:00', '01:13:30', '01:15:00', '01:16:30', '01:18:00'],
    'stop_sequence': [1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21]
}

se_right_diagonal_trip_times_0 = {
    'trip_id': '10-0', # route_id-direction
    'stop_id': ['131-5', '130-5', '141-5', '140-5', '151-5', '150-5', '161-5', '160-5', '170-5', '237-5', 'B-5', '240-5', '187-5', '197-5', '196-5', '207-5', '206-5', '217-5', '216-5', '227-5', '226-5'],
    'arrival_time': ['00:53:00', '00:54:30', '00:56:00', '00:57:30', '00:59:00', '01:00:30', '01:02:00', '01:03:30', '01:05:00', '01:06:30', '01:08:00', '01:09:30', '01:11:00', '01:12:30', '01:14:00', '01:15:30', '01:17:00', '01:18:30', '01:20:00', '01:21:30', '01:23:00'],
    'departure_time': ['00:53:00', '00:54:30', '00:56:00', '00:57:30', '00:59:00', '01:00:30', '01:02:00', '01:03:30', '01:05:00', '01:06:30', '01:08:00', '01:09:30', '01:11:00', '01:12:30', '01:14:00', '01:15:30', '01:17:00', '01:18:30', '01:20:00', '01:21:30', '01:23:00'],
    'stop_sequence': [1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21]
}

se_right_diagonal_trip_times_1 = {
    'trip_id': '10-1', # route_id-direction
    'stop_id': ['226-5', '227-5', '216-5', '217-5', '206-5', '207-5', '196-5', '197-5', '187-5', '240-5', 'B-5', '237-5', '170-5', '160-5', '161-5', '150-5', '151-5', '140-5', '141-5', '130-5', '131-5'],
    'arrival_time': ['00:53:00', '00:54:30', '00:56:00', '00:57:30', '00:59:00', '01:00:30', '01:02:00', '01:03:30', '01:05:00', '01:06:30', '01:08:00', '01:09:30', '01:11:00', '01:12:30', '01:14:00', '01:15:30', '01:17:00', '01:18:30', '01:20:00', '01:21:30', '01:23:00'],
    'departure_time': ['00:53:00', '00:54:30', '00:56:00', '00:57:30', '00:59:00', '01:00:30', '01:02:00', '01:03:30', '01:05:00', '01:06:30', '01:08:00', '01:09:30', '01:11:00', '01:12:30', '01:14:00', '01:15:30', '01:17:00', '01:18:30', '01:20:00', '01:21:30', '01:23:00'],
    'stop_sequence': [1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21]
}

# Use headway to create multiple trips for the study period: 00:00:00 to 08:00:00
base_schedules = [
    sw_left_diagonal_trip_times_0,
    sw_left_diagonal_trip_times_1,
    sw_right_diagonal_trip_times_0,
    sw_right_diagonal_trip_times_1,
    se_left_diagonal_trip_times_0,
    se_left_diagonal_trip_times_1,
    se_right_diagonal_trip_times_0,
    se_right_diagonal_trip_times_1
]

all_schedules_list = []
for schedule_data in base_schedules:
    expanded_schedule = expand_timetable(schedule_data, STUDY_START, STUDY_END, BUS_HEADWAY)
    all_schedules_list.append(expanded_schedule)
final_timetable = pd.concat(all_schedules_list, ignore_index=True)
final_timetable['arrival_time'] = final_timetable['arrival_time'].dt.strftime('%H:%M:%S')
final_timetable['departure_time'] = final_timetable['departure_time'].dt.strftime('%H:%M:%S')
stop_times_fp_df = final_timetable
stop_times_fp_df.to_csv(os.path.join(basic_output_path, "stop_times_fp.txt"), index=False)

In [21]:
# Create trips_fp.txt
trips_fp_df = pd.DataFrame(columns=['trip_id','route_id','service_id','direction_id'])

all_trip_ids = final_timetable['trip_id'].unique().tolist()
for trip_id in all_trip_ids:
    route_id = int(trip_id.split('-')[0])
    direction_id = int(trip_id.split('-')[1])
    new_row = pd.DataFrame({
        'trip_id': [trip_id],
        'route_id': [route_id],
        'service_id': [0],
        'direction_id': [direction_id]
    })
    trips_fp_df = pd.concat([trips_fp_df, new_row], ignore_index=True)

trips_fp_df.to_csv(os.path.join(basic_output_path, "trips_fp.txt"), index=False)